In [1]:
import os
project_cache_dir = "./hf_cache"
os.makedirs(project_cache_dir, exist_ok=True)

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel

# models
model_name = "SpanBERT/spanbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=project_cache_dir,
    lcoal_files_only=False
)
encoder = AutoModel.from_pretrained(
    model_name,
    cache_dir=project_cache_dir,
    local_files_only=False
)
encoder.eval()


/home/dan/Work/utcn/an4/sem1/pso/proj/knowledge-graph-extraction/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [3]:
# gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
print("[Stage 1] Loaded SpanBERT-base model on device:", device)

[Stage 1] Loaded SpanBERT-base model on device: cuda


In [4]:
# test input
text = ("Lionel Messi scored a brilliant goal. "
        "The Argentine forward celebrated with his teammates. "
        "He is considered one of the best players in the world.")
print("[Stage 2] Input text:\n", text)

[Stage 2] Input text:
 Lionel Messi scored a brilliant goal. The Argentine forward celebrated with his teammates. He is considered one of the best players in the world.


In [5]:
# inference
tokens = tokenizer(text, return_tensors="pt")
tokens = {k:v.to(device) for k,v in tokens.items()}

with torch.no_grad():
    outputs = encoder(**tokens)

token_embeddings = outputs.last_hidden_state.squeeze(0)  # [seq_len, hidden_size]
print("\n[Stage 3] Token embeddings shape:", token_embeddings.shape)


[Stage 3] Token embeddings shape: torch.Size([33, 768])


In [14]:
max_span_width = 10
span_indices = []
for i in range(token_embeddings.size(0)):
    for w in range(1, max_span_width+1):
        if i + w <= token_embeddings.size(0):
            span_indices.append((i, i + w - 1))
print("\n[Stage 4] Candidate spans (token indices):\n", span_indices)


[Stage 4] Candidate spans (token indices):
 [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (7, 16), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15), (8, 16), (8, 17), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19), (11, 11),

In [15]:
span_embeddings = torch.stack([token_embeddings[start] + token_embeddings[end] for start, end in span_indices])
print("\n[Stage 5] Span embeddings shape:", span_embeddings.shape)


[Stage 5] Span embeddings shape: torch.Size([285, 768])


In [16]:
scores = torch.matmul(span_embeddings, span_embeddings.T)
scores = torch.tril(scores, diagonal=-1)
print("\n[Stage 6] Antecedent scores matrix shape:", scores.shape)


[Stage 6] Antecedent scores matrix shape: torch.Size([285, 285])


In [17]:
# clustering 
clusters = []
span_to_cluster = {}
for i, (start, end) in enumerate(span_indices):
    if i == 0:
        span_to_cluster[i] = len(clusters)
        clusters.append([(start, end)])
        continue
    antecedents = scores[i,:i]
    if antecedents.max() > .2:
        best = antecedents.argmax().item()
        span_to_cluster[i] = span_to_cluster[best]
        clusters[span_to_cluster[i]].append((start, end))
    else:
        span_to_cluster[i] = len(clusters)
        clusters.append([(start, end)])
        
print("\n[Stage 7] Clusters (token indices):\n", clusters)


[Stage 7] Clusters (token indices):
 [[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (7, 16), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15), (8, 16), (8, 17), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19), (11, 11), (11, 

In [18]:
# conversion back to text
token_list = tokenizer.convert_ids_to_tokens(tokens["input_ids"].squeeze())
clusters_text = []
for cluster in clusters:
    clusters_text.append([" ".join(token_list[start:end+1]) for start, end in cluster])


In [19]:
# output
print("Candidate clusters (resolved mentions):")
for c in clusters_text:
    print(c)


Candidate clusters (resolved mentions):
['[CLS]', '[CLS] lion', '[CLS] lion ##el', '[CLS] lion ##el mess', '[CLS] lion ##el mess ##i', '[CLS] lion ##el mess ##i scored', '[CLS] lion ##el mess ##i scored a', '[CLS] lion ##el mess ##i scored a brilliant', '[CLS] lion ##el mess ##i scored a brilliant goal', '[CLS] lion ##el mess ##i scored a brilliant goal .', 'lion', 'lion ##el', 'lion ##el mess', 'lion ##el mess ##i', 'lion ##el mess ##i scored', 'lion ##el mess ##i scored a', 'lion ##el mess ##i scored a brilliant', 'lion ##el mess ##i scored a brilliant goal', 'lion ##el mess ##i scored a brilliant goal .', 'lion ##el mess ##i scored a brilliant goal . the', '##el', '##el mess', '##el mess ##i', '##el mess ##i scored', '##el mess ##i scored a', '##el mess ##i scored a brilliant', '##el mess ##i scored a brilliant goal', '##el mess ##i scored a brilliant goal .', '##el mess ##i scored a brilliant goal . the', '##el mess ##i scored a brilliant goal . the a', 'mess', 'mess ##i', 'mess ##

In [20]:
for i, (start, end) in enumerate(span_indices):
    span_text = " ".join(token_list[start:end+1])
    if i == 0:
        print(f"{span_text:20} -> None")
        continue
    antecedents = scores[i,:i]
    top_score, top_idx = antecedents.max(0)
    top_span = " ".join(token_list[span_indices[top_idx][0]:span_indices[top_idx][1]+1])
    print(f"{span_text:20} -> {top_span:20} (score={top_score.item():.2f})")

[CLS]                -> None
[CLS] lion           -> [CLS]                (score=48.06)
[CLS] lion ##el      -> [CLS]                (score=49.60)
[CLS] lion ##el mess -> [CLS]                (score=50.55)
[CLS] lion ##el mess ##i -> [CLS] lion ##el      (score=51.91)
[CLS] lion ##el mess ##i scored -> [CLS] lion ##el mess ##i (score=54.01)
[CLS] lion ##el mess ##i scored a -> [CLS] lion ##el mess ##i scored (score=58.94)
[CLS] lion ##el mess ##i scored a brilliant -> [CLS] lion ##el mess ##i scored a (score=54.78)
[CLS] lion ##el mess ##i scored a brilliant goal -> [CLS] lion ##el mess ##i scored (score=56.05)
[CLS] lion ##el mess ##i scored a brilliant goal . -> [CLS] lion ##el mess ##i scored a (score=54.61)
lion                 -> [CLS] lion           (score=50.67)
lion ##el            -> lion                 (score=53.70)
lion ##el mess       -> lion                 (score=62.37)
lion ##el mess ##i   -> lion ##el            (score=52.16)
lion ##el mess ##i scored -> [CLS] lion ##e

In [13]:
# clusters_text
clusters_text = []
for cluster in clusters:
    clusters_text.append([" ".join(token_list[start:end+1]) for start, end in cluster])
print("\n[Final Output] Candidate clusters (resolved mentions ):", clusters_text)

# Only clusters with ≥2 mentions
relevant_clusters = [c for c in clusters_text if len(c) > 1]

print("\n[Relevant Entity Clusters]:")
for c in relevant_clusters:
    print(c)



[Final Output] Candidate clusters (resolved mentions): [['[CLS]', '[CLS] lion', 'lion', 'lion ##el', '##el', '##el mess', 'mess', 'mess ##i', '##i', '##i scored', 'scored', 'scored a', 'a', 'a brilliant', 'brilliant', 'brilliant goal', 'goal', 'goal .', '.', '. the', 'the', 'the a', 'a', 'a ##rgent', '##rgent', '##rgent ##ine', '##ine', '##ine forward', 'forward', 'forward celebrated', 'celebrated', 'celebrated with', 'with', 'with his', 'his', 'his teammates', 'teammates', 'teammates .', '.', '. he', 'he', 'he is', 'is', 'is considered', 'considered', 'considered one', 'one', 'one of', 'of', 'of the', 'the', 'the best', 'best', 'best players', 'players', 'players in', 'in', 'in the', 'the', 'the world', 'world', 'world .', '.', '. [SEP]', '[SEP]']]

[Relevant Entity Clusters]:
['[CLS]', '[CLS] lion', 'lion', 'lion ##el', '##el', '##el mess', 'mess', 'mess ##i', '##i', '##i scored', 'scored', 'scored a', 'a', 'a brilliant', 'brilliant', 'brilliant goal', 'goal', 'goal .', '.', '. the'